<h1>Задание 4</h1>

4.       Реализовать алгоритм автоматического реферирования (квазиреферирование) на основе статистического подхода, алгоритм приведен ниже. Входные данные: исходный текст и коэффициент сжатия. Единицей реферирования (фрагментом) должно являться предложение. Выходные данные: список ключевых слов с весами, список предложений с весами, текст реферата. (5 баллов)

Алгоритм:

1.       Разбить текст на предложения.

2.       Разбить текст на слова произвести их нормализацию.

3.       Удалить стоп-слова.

4.       Подсчитать веса слов (частота слова в тексте или TF-IDF, если выполнялось задание 2).

5.       Определить веса предложений, рассчитанный как сумма весов, входящих в предложение слов.

6.       Отсортировать предложения по убыванию веса.

7.       В отсортированном списке оставить те предложений, которые входят в задаваемый процент сжатия. Например, если процент сжатия 10%, а в исходном тексте 50 предложений, то в реферате будет 5 предложений.

8.       Сформировать текст реферата из отобранных на предыдущем шаге предложений в порядке, котором они встретились в исходном тексте.

In [171]:
import nltk
import pandas as pd
import math
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk import FreqDist
import re

#Create and stopwords list
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'какой-то', 'который'])

punctuation = punctuation + '...' + '—' + '«' + '»' + '„' + '“' + '№'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~...—«»„“№'

In [222]:
import os
import glob

def clean_folder():
    files = glob.glob('4_result/*')
    for f in files:
        os.remove(f)

In [172]:
def get_text(file_name):
    txt = ''
    with open(file_name) as f:
        for line in f:
            line = line.rstrip("\r\n")
            line = line.rstrip(".—")
            txt += line + ' '
    return(txt)

In [173]:
def get_sentences(text):
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(text)])
    corpus = []
    for s in sentences:
        corpus.append(s)
    return(corpus)

In [174]:
#Preprocess function
def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [ morph.parse(word.lower())[0].normal_form for word in tokens if (word != " ") and (word not in punctuation)]
    filtered_tokens = []
    for word in lemmatized_tokens:
        if (word not in russian_stopwords) and (word != " ") and (word not in punctuation):
            filtered_tokens.append(word)
    
    return(filtered_tokens) 

In [175]:
def frequency_of_words(txt):
    frequency_distribution = FreqDist(txt)
    word = []
    frequency = []
    for key in frequency_distribution:
            word.append(key)
            frequency.append(int(frequency_distribution[key]))
    return (word, frequency)

In [176]:
def compute_tf(text):
    frequency_distribution = FreqDist(text)
    
    word = []
    tf = []
    if len(text) > 0:
        for key in frequency_distribution:
            word.append(key)
            tf.append(int(frequency_distribution[key])/int(len(text)))
        
    return word, tf

In [177]:
def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [178]:
def compute_words_weight(corpus, corpus_of_tokens):
    corpus = preprocess_text(corpus)
    #Определить вес предложения
    idf = []
    word_weight = []
    words, tf = compute_tf(corpus)
    for w in words:
        w_idf = compute_idf(w, corpus_of_tokens)
        idf.append(w_idf)
        
    for i in range (0, len(words)):
        word_weight.append(tf[i]*idf[i])

    return(words, word_weight)

In [179]:
def compute_proposal_weight(corpus, corpus_of_tokens):
    corpus = preprocess_text(corpus)
    #Определить вес предложения
    idf = []
    proposal_weight = 0
    words, tf = compute_tf(corpus)
    for w in words:
        w_idf = compute_idf(w, corpus_of_tokens)
        idf.append(w_idf)
        
    for i in range (0, len(words)):
        proposal_weight += tf[i]*idf[i]

    return(proposal_weight)

In [195]:
def write_to_txt_file(text):
    with open('4_result/referat.txt','w') as f:
        f.write(text)
        
    f.close()

In [203]:
def write_to_csv_file(df, file_name):
    file_path = '4_result/' + file_name + '.csv'
    df.to_csv(file_path, sep='\t', encoding='utf-8')

In [229]:
def main (file_name, procent):
    clean_folder()
    
    txt = get_text(file_name)
    
    #Разбить текст на предложения.
    corpus = get_sentences(txt)
    
    #Разбить текст на слова произвести их нормализацию + Удалить стоп-слова
    txt = preprocess_text(txt)
    
    #Подсчитать веса слов (частота слова в тексте или TF-IDF, если выполнялось задание 2).
    #Cоздать корпус текстов/токенов текстов
    corpus_of_tokens = []
    for sentence in corpus:
        tokens = preprocess_text(sentence)
        corpus_of_tokens.append(tokens)
        
    order_num = []
    sentences = []
    tf_idf = [] 
    word = []
    df1 = pd.DataFrame(columns=['order_s_in_text', 'sentence', 'word', 'tf_idf'])
    i = 1
    for sentence in corpus:
        words, weights = compute_words_weight(sentence, corpus_of_tokens)
        order_num.append(i)
        sentences.append(sentence)
        tf_idf.append(weights)
        word.append(words)
        i += 1

    df1['order_s_in_text'] = order_num
    df1['sentence'] = sentences
    df1['word'] = word
    df1['tf_idf'] = tf_idf
    write_to_csv_file(df1, 'words_weight')
    
    #Определить веса предложений, рассчитанный как сумма весов, входящих в предложение слов.
    order_num = []
    tf_idf = []    
    df = pd.DataFrame(columns=['order_in_text', 'sentences', 'tf_idf'])
    

    i = 1
    for sentence in corpus:
        order_num.append(i)
        tf_idf.append(compute_proposal_weight(sentence, corpus_of_tokens))
        i += 1

    df['order_in_text'] = order_num
    df['sentences'] = corpus
    df['tf_idf'] = tf_idf
    
    #Отсортировать предложения по убыванию веса.
    #В отсортированном списке оставить те предложений, которые входят в задаваемый процент сжатия. 
    procent = round(df.shape[0] * procent)
    result = pd.DataFrame(columns=['order_in_text', 'sentences', 'tf_idf'])
    result = df.sort_values('tf_idf', ascending = False)
    write_to_csv_file(result, 'sentences_weight')
    
    
    
    #Сформировать текст реферата из отобранных на предыдущем шаге предложений в порядке, котором они встретились в исходном тексте.
    result2 = df.sort_values('tf_idf', ascending = False).head(procent)
    result2 = result2.sort_values('order_in_text')
    result2['sentences'] = result2['sentences']+'.'
    referat = result2['sentences']
    referat = ' '.join(referat)
    write_to_txt_file(referat)
    
    return df1, result, referat

In [236]:
words_weights, sentences_weights, referat = main('text.txt', 0.1)
words_weights.head(100)

,order_s_in_text,sentence,word,tf_idf
0,1,"В середине августа, перед рождением молодого м...","[середина, август, рождение, молодой, месяц, н...","[0.17743380315571244, 0.23120072656617544, 0.2..."
1,2,То по целым суткам тяжело лежал над землею и м...,"[целый, сутки, тяжело, лежать, земля, море, гу...","[0.11797702619766355, 0.17680055560942826, 0.1..."
2,3,"То с утра до утра шел не переставая мелкий, ка...","[утро, идти, переставать, мелкий, водяной, пыл...","[0.284692573652242, 0.1033798294843187, 0.1581..."
3,4,"То задувал с северо-запада, со стороны степи с...","[задувать, северо-запад, сторона, степь, свире...","[0.09107907410182668, 0.09107907410182668, 0.0..."
4,5,Несколько рыбачьих баркасов заблудилось в море...,"[несколько, рыбачий, баркас, заблудиться, море...","[0.06642855859698812, 0.09326136033435514, 0.1..."
...,...,...,...,...
95,96,"прохлада — Мне все равно, я все люблю, — ответ...","[всё, прохлада, равно, любить, ответить, анна]","[0.3527878382051817, 0.386368492813757, 0.3182..."
96,97,"— А больше всего я люблю мою сестренку, мою бл...","[большой, весь, любить, сестрёнка, благоразумн...","[0.24568842138633754, 0.22702612814568215, 0.2..."
97,98,Нас ведь только двое на свете Она обняла старш...,"[щека, двое, свет, обнять, старший, сестра, пр...","[0.6321220476601545, 0.37570118067003505, 0.24..."
98,99,"И вдруг спохватилась — Нет, какая же я глупая","[спохватиться, глупый]","[1.5028047226801402, 1.1533197205121308]"


In [237]:
sentences_weights.head(100)

,order_in_text,sentences,tf_idf
175,176,киселе,3.005609
155,156,Ступай,3.005609
754,755,юноше,3.005609
747,748,Через жандармов,3.005609
742,743,Он мне приятель по клубу,3.005609
...,...,...,...
671,672,Их сумасбродные прихоти и капризы станут для н...,2.719609
5,6,По размякшему шоссе без конца тянулись ломовые...,2.719089
306,307,"Теперь во мне осталось только благоговение, ве...",2.715549
29,30,Через несколько минут у дачных ворот круто ост...,2.711656


In [238]:
referat

'То с утра до утра шел не переставая мелкий, как водяная пыль, дождик, превращавший глинистые дороги и тропинки в сплошную густую грязь, в которой увязали надолго возы и экипажи. То задувал с северо-запада, со стороны степи свирепый ураган; от него верхушки деревьев раскачивались, пригибаясь и выпрямляясь, точно волны в бурю, гремели по ночам железные кровли дач, и казалось, будто кто-то бегает по ним в подкованных сапогах, вздрагивали оконные рамы, хлопали двери, и дико завывало в печных трубах. По размякшему шоссе без конца тянулись ломовые дроги, перегруженные всяческими домашними вещами: тюфяками, диванами, сундуками, стульями, умывальниками, самоварами. На обсохших сжатых полях, на их колючей желтой щетине заблестела слюдяным блеском осенняя паутина. Через несколько минут у дачных ворот круто остановился изящный автомобиль-карета, и шофер, ловко спрыгнув с сиденья, распахнул дверцу Сестры радостно поцеловались. — Ты всегда придумаешь. Это там, за лесничеством, выше водопада. Хорош